## Carga informacion

In [1]:
mes = "Octubre"

In [2]:
import glob
import os
import pandas as pd
import warnings
from pathlib import Path

# Ocultar warnings de openpyxl sobre headers/footers
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# ==========
# 1) Unir los 8 Excels de septiembre
# ==========
ruta_ns = r"/home/donsson/proyectos/INDICADOR NS/dataset/ns_octubre"
archivos_ns = glob.glob(os.path.join(ruta_ns, "*.xlsx"))

if not archivos_ns:
    raise FileNotFoundError(f"No se encontró ningún archivo en {ruta_ns}")

# Unir todos los Excels en un solo DataFrame
df_unido = pd.concat([pd.read_excel(f) for f in archivos_ns], ignore_index=True)

print(f"✅ Se unieron {len(archivos_ns)} archivos de Octubre con {df_unido.shape[0]} filas totales")

# ==========
# 2) Cargar inventario limpio
# ==========
ruta_inventario = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

if not os.path.exists(ruta_inventario):
    raise FileNotFoundError(f"No existe el archivo {ruta_inventario}")

df_inventario = pd.read_excel(ruta_inventario)

print(f"📦 Inventario cargado con {df_inventario.shape[0]} filas desde: {ruta_inventario}")



✅ Se unieron 8 archivos de Octubre con 39000 filas totales
📦 Inventario cargado con 14378 filas desde: /home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx


In [3]:
df_unido.sample(10)

,Sucursal,Marca,Clase,Producto,Descripcion,Enero nv,Febrero nv,Marzo nv,Abril nv,Mayo nv,...,Marzo vp,Abril vp,Mayo vp,Junio vp,Julio vp,Agosto vp,Septiembre vp,Octubre vp,Noviembre vp,Diciembre vp
13725,SUCURSAL CALI,DONSSON,C,DAE04999025,[DAE04999025] DA4999 FILTRO AIRE SEGURIDAD - H...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31406,SUCURSAL BUCARAMANGA,MOBIL IMPORTADO,C,ALT15W40131,[ALT15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
18071,SUCURSAL MEDELLIN,BALDWIN,C,BLS00218125,[BLS00218125] GS218 FILTRO ACEITE DUAL CATERPI...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8010,SUCURSAL BARRANQUILLA,BALDWIN,C,BLS00608125,[BLS00608125] GS608 FILTRO ACEITE BY PASS BOBC...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16737,SUCURSAL MEDELLIN,OTROS,C,DAE06763136,[DAE06763136] GA763 FILTRO AIRE POCKET FELICIA...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19869,SUCURSAL VALLADOLID,AUT*PARTS,B,DCS00605189,[DCS00605189] GS605 FILTRO COMBUSTIBLE SEPARAD...,1.0,0.667,0.389,1.0,0.245,...,11.0,NaN,71.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN
23467,SUCURSAL VALLADOLID,DONSSON,C,DAE05387025,[DAE05387025] GA387 FILTRO AIRE MO (025 DA5387),1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6258,SUCURSAL BARRANQUILLA,RACOR USA,C,DCX00187138,[DCX00187138] GX187B SEPARADOR COMB. RACOR (In...,1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38503,SUCURSAL CALLE 6,DONSSON,C,DAE02322025,"[DAE02322025] DA2322 FILTRO AIRE JONH DEERE,MA...",1.0,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2148,SUCURSAL NORTE,OTROS,C,DAE06725136,[DAE06725136] GA725 FILTRO AIRE NISSAN SENTRA ...,NaN,1.000,1.000,1.0,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [4]:
df_sin_clase = df_unido[df_unido["Clase"]=="SIN CLASE X SUCURSAL"]  # identificar productos sin categoria y revisar si son nuevos

df_sin_clase = df_sin_clase.groupby("Descripcion")["Producto"].count()
df_sin_clase.sample()

Descripcion
[ACUEDUCTO Y ALCANTARILLADO] ACUEDUCTO Y ALCANTARILLADO    8
Name: Producto, dtype: int64

## Limpieza

In [5]:
from datetime import datetime
import re

def limpiar_y_transformar(df_unido):
    # 1. Obtener el mes actual en español manualmente
    meses_es = [
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    ]
    mes_actual = meses_es[datetime.now().month - 1]

    # 2. Quitar columnas desde el mes siguiente al actual
    meses_nv = [f"{mes.lower()} nv" for mes in meses_es]
    meses_vp = [f"{mes.lower()} vp" for mes in meses_es]

    idx_mes_actual = meses_es.index(mes_actual)

    # Columnas a eliminar: desde el mes siguiente hasta diciembre
    meses_a_borrar = meses_nv[idx_mes_actual+1:] + meses_vp[idx_mes_actual+1:]
    columnas_a_eliminar = [col for col in df_unido.columns if col.lower() in meses_a_borrar]
    df_unido = df_unido.drop(columns=columnas_a_eliminar, errors='ignore')

    # 3. Clasificación categoría
    marcas_filtros = [ "BALDWIN", "DONSSON", "AUT*PARTS", "RAMA", "RACOR USA", "RACOR BRASIL"
    ]

    #marcas_aceite_lubricantes = []

    def clasificar_categoria(marca):
        marca = str(marca).upper().strip()
        if marca == "OTROS":
            return "Otros"
        elif marca in marcas_filtros:
            return "Filtro"
        else:
            return "Aceite/Lubricante"

    df_unido["categoria_producto"] = df_unido["Marca"].apply(clasificar_categoria)



    # 4. Extraer código interno        
    def extraer_codigo(descripcion, categoria):
        if not isinstance(descripcion, str):
            return None

    # Para Aceite/Lubricante, tomar directamente lo que está dentro de los corchetes
        if categoria == "Aceite/Lubricante":
            match = re.search(r"\[([A-Z0-9\-]+)\]", descripcion)
            if match:
                return match.group(1)

    # Para Filtros, aplicar la lógica anterior

    
        elif categoria == "Filtro" or categoria == "Otros":
            match = re.search(r"\[([A-Z0-9\-]+)\]\s+(\S+)", descripcion)
            if match:
                codigo_base = match.group(1)
                palabra_siguiente = match.group(2)
            
            # Tomar los últimos 3 dígitos (no caracteres): buscar solo dígitos desde el final
            ultimos_digitos = ''.join(re.findall(r'\d', codigo_base))[-3:]
            return f"{ultimos_digitos} {palabra_siguiente}"

        

    df_unido["Codigo_interno"] = df_unido.apply(
    lambda row: extraer_codigo(row["Descripcion"], row["categoria_producto"]),
    axis=1)
    df_unido['Codigo_Numerico'] = df_unido['Codigo_interno'].str.split(' ').str[0]


    # 5. Definir las listas de códigos numéricos para filtros
    importacion = ['189', '125', '138', '137', '222', '225']
    nacional = ['004', '190', '005']
    fabricado = ['025']

    # 5. Crear una función de clasificación que considera la categoría y el código numérico
    def clasificar_producto(row):
        # Si la categoría es "Filtro", usamos las listas para clasificar
        if row['categoria_producto'] == 'Filtro':
            if row['Codigo_Numerico'] in importacion:
                return 'Importacion'
            elif row['Codigo_Numerico'] in nacional:
                return 'Nacional'
            elif row['Codigo_Numerico'] in fabricado:
                return 'Fabricado'
            else:
                return 'Desconocido' # Si es un filtro con un código no listado
            # Para cualquier otra categoría (Aceite/Lubricante, etc.), clasificar como "Nacional"
        else:
            return 'Nacional'
        
    df_unido['Tipo_origen'] = df_unido.apply(clasificar_producto, axis=1)

                                

    # 6. Eliminar productos con clase "SIN CLASE X SUCURSAL" y aquellos que no tienen marca
     
    df_unido = df_unido[~(df_unido['Marca'].isna() | (df_unido['Marca'].str.strip() == ''))]

    df_unido = df_unido[df_unido["Clase"] != "SIN CLASE X SUCURSAL"]

    df_unido["Sucursal"] = df_unido["Sucursal"].replace("PRINCIPAL COTA", "MOSTRADOR COTA") #Correccion bodega 8 esatba como principal cota
    df_unido["Sucursal"] = df_unido["Sucursal"].str.replace("Sucursal ", "", regex=False) #Dejar sin la palabra sucursal




    # 7. Agregar columna con la meta de nivel de servicio según tipo/clase
    metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.95,
        'C': 0.50
    }

    # Normalizar la columna de clase por si hay espacios o minúsculas
    df_unido["Clase"] = df_unido["Clase"].str.strip().str.upper()

    # Crear columna con la meta
    df_unido["meta_ns"] = df_unido["Clase"].map(metas_nv)

    # 8. Comparar si se cumplió la meta
    # Asegúrate de tener una columna real con el nivel de servicio (ajusta el nombre si es otro)
    if "nivel_servicio" in df_unido.columns:
        df_unido["cumple_meta"] = df_unido["nivel_servicio"] >= df_unido["meta_ns"]




    return df_unido




In [6]:
df_limpio = limpiar_y_transformar(df_unido)
df_limpio = df_limpio[['Sucursal' ,'categoria_producto','Producto', 'Marca' , 'Clase', 'Codigo_interno' , f'{mes} nv', f'{mes} vp','meta_ns','Tipo_origen','Descripcion']]
df_limpio.head()

,Sucursal,categoria_producto,Producto,Marca,Clase,Codigo_interno,Octubre nv,Octubre vp,meta_ns,Tipo_origen,Descripcion
0,SUCURSAL NORTE,Otros,DAC00246020,OTROS,C,020 DAC246KIT,NaN,2.0,0.5,Nacional,[DAC00246020] DAC246KIT FILTRO CABINA JUEGO - ...
1,SUCURSAL NORTE,Otros,DAC00245020,OTROS,C,020 DAC245KIT,1.0,NaN,0.5,Nacional,[DAC00245020] DAC245KIT FILTRO CABINA JUEGO - ...
2,SUCURSAL NORTE,Filtro,DAB09280025,DONSSON,C,025 DA9280,1.0,NaN,0.5,Fabricado,[DAB09280025] DA9280 FILTRO AIRE SEGURIDAD - S...
3,SUCURSAL NORTE,Filtro,DAB08280025,DONSSON,C,025 DA8280,1.0,NaN,0.5,Fabricado,[DAB08280025] DA8280 FILTRO AIRE - SCANIA EN D...
4,SUCURSAL NORTE,Filtro,DHE01054189,AUT*PARTS,C,189 G1054,1.0,NaN,0.5,Importacion,[DHE01054189] G1054 FILTRO HIDRAULICO DIRECCIO...


In [7]:
df_limpio.groupby(['Sucursal','Clase'])[f'{mes} vp'].sum()

Sucursal               Clase
MOSTRADOR COTA         A         213.0
                       AAA        68.0
                       B         434.0
                       C        1744.0
SUCURSAL BARRANQUILLA  A         308.0
                       AAA        29.0
                       B         306.0
                       C         922.0
SUCURSAL BUCARAMANGA   A          62.0
                       AAA        22.0
                       B         101.0
                       C         510.0
SUCURSAL CALI          A         250.0
                       AAA       543.0
                       B         296.0
                       C         710.0
SUCURSAL CALLE 6       A         408.0
                       AAA        52.0
                       B         266.0
                       C         599.0
SUCURSAL MEDELLIN      A          70.0
                       AAA         8.0
                       B         116.0
                       C         588.0
SUCURSAL NORTE         A         19

### Puedo ver las ventas perdidas de todos los meses que han trasncurrido

In [8]:
df_completo = limpiar_y_transformar(df_unido)
df_completo_vp = df_completo[['Sucursal' ,'categoria_producto', 'Marca' , 'Clase', 'Codigo_interno' ,'Enero vp','Febrero vp','Marzo vp','Mayo vp','Abril vp','Junio vp','Julio vp','Agosto vp','Septiembre vp','Octubre vp','meta_ns','Tipo_origen','Descripcion']]
df_largo = df_completo_vp.melt(
    id_vars=['Sucursal', 'categoria_producto', 'Marca', 'Clase', 'Codigo_interno', 'meta_ns', 'Tipo_origen', 'Descripcion'],
    value_vars=['Enero vp', 'Febrero vp', 'Marzo vp', 'Abril vp', 'Mayo vp', 'Junio vp', 'Julio vp', 'Agosto vp', 'Septiembre vp','Octubre vp'],
    var_name='Mes',
    value_name='Ventas Perdidas'
)


# Quitar el " vp" del final
df_largo['Mes'] = df_largo['Mes'].str.replace(' vp', '', regex=False)

# Diccionario para ordenar meses
orden_meses = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Columna auxiliar de orden
df_largo['Mes_num'] = df_largo['Mes'].map(orden_meses)


# Primero eliminar filas que no aportan nada
# Agrupamos por producto y verificamos si TODOS los valores de "Ventas Perdidas" son NaN o 0
mask = df_largo.groupby("Codigo_interno")["Ventas Perdidas"].transform(
    lambda x: x.notna().any() and x.fillna(0).sum() > 0
)

# Filtrar solo los productos que sí tienen al menos una venta perdida
df_filtrado = df_largo[mask]

df_agosto = df_filtrado[df_filtrado["Mes"]==f"{mes}"]

df_agosto.to_excel(f"vp_{mes}.xlsx")

# Guardar para BI
df_filtrado.to_excel("v_perdidas/ventas_perdidas_general.xlsx", index=False)


# 2 Min

## Unir DF costo

In [9]:
df_inv_costos = df_inventario[['Codigo_interno', 'Sucursal', 'Costo unitario']].drop_duplicates()

df_merged = df_limpio.merge( 
    df_inv_costos,
    on = ['Codigo_interno','Sucursal'], 
    how='left'
)

df_merged['costo_vp'] = df_merged[f'{mes} vp'] * df_merged['Costo unitario']

df_limpio= df_merged

In [10]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38656 entries, 0 to 38655
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sucursal            38656 non-null  object 
 1   categoria_producto  38656 non-null  object 
 2   Producto            38656 non-null  object 
 3   Marca               38656 non-null  object 
 4   Clase               38656 non-null  object 
 5   Codigo_interno      38656 non-null  object 
 6   Octubre nv          36774 non-null  float64
 7   Octubre vp          2844 non-null   float64
 8   meta_ns             38656 non-null  float64
 9   Tipo_origen         38656 non-null  object 
 10  Descripcion         38656 non-null  object 
 11  Costo unitario      8741 non-null   float64
 12  costo_vp            663 non-null    float64
dtypes: float64(5), object(8)
memory usage: 3.8+ MB


In [11]:


# Suponiendo que ya tienes tu df_limpio
# df_limpio = cargar_datos(str(ruta_carpeta))

# 1) Pedir el mes al usuario
mes = input("\nIngrese el mes (ejemplo: julio): ").strip().lower()

# 2) Construir la ruta completa del archivo
nombre_archivo = Path("informes") / f"datos_{mes}.xlsx"

# 3) Guardar el DataFrame en la carpeta 'informes'
df_limpio.to_excel(nombre_archivo, index=False)

print(f"\n✓ Archivo guardado como {nombre_archivo}")



✓ Archivo guardado como informes/datos_octibre.xlsx


# Indicador NS

In [12]:
metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.90,
        'C': 0.50
    }

def calcular_nivel_servicio(df, mes=None):
    # Paso 1: Identificar columnas de nivel de servicio
    columnas_nv = [col for col in df.columns if col.endswith('nv')]
    
    # Paso 2: Ordenar las columnas por orden real de meses
    orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                   'Julio','Agosto','Septiembre','Octubre']
    
    columnas_nv_ordenadas = sorted(
        columnas_nv,
        key=lambda x: orden_meses.index(x.split()[0])
    )

    # Paso 3: Si no se especifica el mes, detectar el último con al menos 80% de datos
    if mes is None:
        mes_final = None
        for col in reversed(columnas_nv_ordenadas):
            if df[col].notna().mean() > 0.8:
                mes_final = col
                break
        if mes_final is None:
            raise ValueError("No hay columnas de nivel de servicio con suficientes datos.")
    else:
        mes_final = next((col for col in columnas_nv if col.startswith(mes)), None)
        if mes_final is None:
            raise ValueError(f"No se encontró una columna de nivel de servicio para el mes '{mes}'.")

    # Paso 4: Calcular promedio por sucursal y clase
    resultado = (
        df.groupby(['Sucursal', 'Clase'])[mes_final]
        .mean()
        .reset_index()
        .rename(columns={mes_final: 'Nivel_Servicio'})
    )

    # Paso 5: Evaluar contra la meta según clase
    resultado['Meta'] = resultado['Clase'].map(metas_nv)
    resultado['Cumple'] = resultado['Nivel_Servicio'] >= resultado['Meta']
    resultado['Mes'] = mes_final.split()[0]

    return resultado


In [13]:
# Si quieres que el sistema lo escoja automáticamente:
nivel_servicio = calcular_nivel_servicio(df_limpio)

# O si tú decides el mes (por ejemplo, "Junio"):
nivel_servicio = calcular_nivel_servicio(df_limpio, mes="Octubre")

nivel_servicio

,Sucursal,Clase,Nivel_Servicio,Meta,Cumple,Mes
0,MOSTRADOR COTA,A,0.964570,0.980,False,Octubre
1,MOSTRADOR COTA,AAA,0.981571,0.999,False,Octubre
2,MOSTRADOR COTA,B,0.950905,0.900,True,Octubre
3,MOSTRADOR COTA,C,0.989343,0.500,True,Octubre
4,SUCURSAL BARRANQUILLA,A,0.969015,0.980,False,Octubre
5,SUCURSAL BARRANQUILLA,AAA,0.993156,0.999,False,Octubre
6,SUCURSAL BARRANQUILLA,B,0.953277,0.900,True,Octubre
7,SUCURSAL BARRANQUILLA,C,0.995935,0.500,True,Octubre
8,SUCURSAL BUCARAMANGA,A,0.995527,0.980,True,Octubre
9,SUCURSAL BUCARAMANGA,AAA,0.996923,0.999,False,Octubre


In [14]:
df_largo.head()

,Sucursal,categoria_producto,Marca,Clase,Codigo_interno,meta_ns,Tipo_origen,Descripcion,Mes,Ventas Perdidas,Mes_num
0,SUCURSAL NORTE,Otros,OTROS,C,020 DAC246KIT,0.5,Nacional,[DAC00246020] DAC246KIT FILTRO CABINA JUEGO - ...,Enero,NaN,1
1,SUCURSAL NORTE,Otros,OTROS,C,020 DAC245KIT,0.5,Nacional,[DAC00245020] DAC245KIT FILTRO CABINA JUEGO - ...,Enero,NaN,1
2,SUCURSAL NORTE,Filtro,DONSSON,C,025 DA9280,0.5,Fabricado,[DAB09280025] DA9280 FILTRO AIRE SEGURIDAD - S...,Enero,NaN,1
3,SUCURSAL NORTE,Filtro,DONSSON,C,025 DA8280,0.5,Fabricado,[DAB08280025] DA8280 FILTRO AIRE - SCANIA EN D...,Enero,NaN,1
4,SUCURSAL NORTE,Filtro,AUT*PARTS,C,189 G1054,0.5,Importacion,[DHE01054189] G1054 FILTRO HIDRAULICO DIRECCIO...,Enero,NaN,1
